In [12]:
import numpy as np

# !pip install scikit-learn
from sklearn.model_selection import KFold,cross_val_score,train_test_split
import matplotlib.image as Image
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.empty_cache()



In [13]:
torch.cuda.is_available()


True

In [14]:
torch.cuda.current_device()


0

In [4]:
torch.cuda.get_device_name(0)

'GeForce RTX 3080'

In [5]:
cuda = torch.device('cuda')

In [6]:
# m_check = ModelCheckpoint("model/--{epoch:02d}--{val_loss:.4f}.hdf5", monitor = 'val_loss',save_best_only=True)


In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.n=1
        self.conv1 = nn.Conv2d(3,self.n*16,(3,3))
        self.pool = nn.MaxPool2d(2,2)
        self.drop = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(self.n*16,self.n*8,(3,3))
        self.conv3 = nn.Conv2d(self.n*8,self.n*4,(3,3))
        self.conv4 = nn.Conv2d(self.n*4,self.n*2,(3,3))
        self.conv5 = nn.Conv2d(self.n*2,self.n,(3,3))
        self.flat = nn.Flatten()
        # self.fc1 = nn.Linear()
        # self.fc2 = nn.Linear()
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.flat(x)
        print(x.size())
        return x



        

In [8]:
net = Net()
net = net.cuda()


In [9]:
seed = np.random.seed(7)

kf = KFold(n_splits=3, shuffle=True,random_state=seed)


In [10]:
x = np.load("../train.npy").reshape(-1,3,112,150)
y = np.load("y.npy")

x_train,x_test , y_train,y_test = train_test_split(x,y,shuffle=True , random_state=seed)

x_train = torch.from_numpy(x_train).float().cuda()
y_train = torch.from_numpy(y_train).float().cuda()

In [11]:
net.forward(x_train)

RuntimeError: CUDA out of memory. Tried to allocate 840.00 MiB (GPU 0; 10.00 GiB total capacity; 7.18 GiB already allocated; 311.05 MiB free; 7.19 GiB reserved in total by PyTorch)

In [4]:
def create_model():
    model = Sequential()
    model.add(Conv2D(200,(3,3),padding='same',activation='relu',input_shape=(112,150,3)))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(200,(3,3),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(3,3),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(3,3),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(2,2),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(100,(2,2),padding='same',activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(2,2))

    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(100,activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(9,activation="softmax"))

    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

    return model


In [ ]:
model = create_model()
for train_i,test_i in kf.split(x):
    train_x,train_y = x[train_i],y[train_i]
    test_x, test_y = x[test_i], y[test_i]

    model.fit(train_x,train_y,batch_size=30,epochs=100,validation_split=0.25,callbacks=[m_check])
    score = model.evaluate(test_x,test_y)
    print(score)

Train on 2250 samples, validate on 750 samples
Epoch 1/100


In [ ]:
predy = model.evaluate(x_test,y_test)
print(predy)